In [1]:
import numpy as np
import pennylane as qml
from pennylane import X, Y, Z, I
from kak_tools import identify_algebra, split_pauli_algebra, map_simple_to_irrep, map_irrep_to_matrices

In [2]:
def close_and_split(gens, verbose=True):
    gens = [list(op.pauli_rep)[0] for op in gens] # Map generators from Operator to PauliWord
    dla = qml.lie_closure(gens, pauli=True)
    dla = [list(op)[0] for op in dla] # Map dla from PauliSentence to PauliWord
    np.random.shuffle(dla)
    sub_dlas = split_pauli_algebra(dla, verbose=verbose)
    return dla, sub_dlas

In [3]:
# Test with so(2n)
n = 8
m = 7
gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
gens += [X(w) @ Y(w+1) for w in range(n, m+n-1)]
dla, sub_dlas = close_and_split(gens)
# print(sub_dlas[0])
# print(sub_dlas[1])
identifiers = identify_algebra(sub_dlas, verbose=True)
print(identifiers)

Found 2 components with dimensions [21, 120].
Dimension of component: 21.
Component 0 can be one of the following:
so(7)
sp(3)
Dimension of component: 120.
Component 1 can be one of the following:
so(16)
8 copies of so(6)
[[(1, 'so', 7), (1, 'sp', 3)], [(1, 'so', 16), (8, 'so', 6)]]


In [4]:
# # Test with random generators
# def make_random_semisimple_algebra(num_wires=5, num_gens=7, seed=None):
#     rng = np.random.default_rng(seed=seed)
#     failure = True
#     while failure:
#         words = rng.choice([I, X, Y, Z], replace=True, size=(num_gens, num_wires))
#         gens = [qml.prod(*(P(w) for w, P in enumerate(word))) for word in words]
#         dla, sub_dlas = close_and_split(gens, verbose=False)
#         dims = [len(sub) for sub in sub_dlas]
#         if 1 not in dims:
#             failure = False
#     return dla
    
# num_wires = 8
# num_gens = 10
# seed = 2
# gens = make_random_semisimple_algebra(num_wires, num_gens, seed=seed)
# # print(f"Max DLA dimension: {4**num_wires-1}")
# dla, sub_dlas = close_and_split(gens)
# # print(sub_dlas)
# # identifiers = identify_algebra(sub_dlas, verbose=True)

## Map to irrep and associated matrices

In [5]:
import numpy as np
from pennylane import X, Y, Z, I, lie_closure
from kak_tools import map_simple_to_irrep, map_irrep_to_matrices

n = 5 # Number of qubits
n_so = 2 * n # The "n" in so(n)
sub_hor_size = 3 * n # Number of random generators that we demand to be mapped to horizontal space (Hamiltonian terms in app)

so_dim = (n_so**2-n_so) // 2 # dimension of so(n_so) = so(2n)
print(f"{n=}; so(2n) dim: {so_dim}")

gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)] # Generators as in FDHS paper
gens = [next(iter(op.pauli_rep)) for op in gens] # Map from qml.operation.Operator to qml.pauli.PauliWord

dla = lie_closure(gens, pauli=True) # Compute Lie closure
dla = [next(iter(op)) for op in dla] # The Lie closure returns PauliSentences, but we want PauliWord's again.
assert len(dla) == so_dim # Assert that we indeed got so(n_so) = so(2n)

def theta(pw):
    """Concurrence ("number of Ys") involution to be applied to PauliWord instances."""
    return -(-1) ** sum(p=="Y" for p in pw.values())

# Make sure we're not trying to squeeze too many operators into the horizontal space (consistency check)
if sub_hor_size > (n_so//2) ** 2:
    raise ValueError("Not enough room in the horizontal space")

# Generate random horizontal Pauli words
hor_gens = np.random.choice([op for op in dla if theta(op) == -1], replace=False, size=sub_hor_size)

n=5; so(2n) dim: 45


In [6]:
# Map index pairs corresponding to E_{ij} matrices to operators in the DLA
mapping, signs = map_simple_to_irrep(dla, hor_gens, n=n_so, invol_type="BDI")
ex_op = mapping[(0, 4)]
print(f"E_{{0, 4}} is mapped to {ex_op}")
# Get rid of the index pairs and map operators in the DLA to matrices directly
matrix_map = map_irrep_to_matrices(mapping, signs, n_so, invol_type="BDI")
print(f"The same Pauli word ({ex_op}) is mapped to the matrix")
matrix_map[ex_op]

E_{0, 4} is mapped to Y(0) @ Z(1) @ Z(2) @ X(3)
The same Pauli word (Y(0) @ Z(1) @ Z(2) @ X(3)) is mapped to the matrix


array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [7]:
H_coeffs = np.random.normal(0, 1., sub_hor_size) # Make up some Hamiltonian coefficients
H = np.tensordot(H_coeffs, np.stack([matrix_map[op] for op in hor_gens]), axes=[[0], [0]]) # Compute Hamiltonian matrix

In [8]:
np.round(H, 1) # Print Hamiltonian matrix

array([[ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.6+0.j,
         0.5+0.j,  0.3+0.j,  0. +0.j, -0.6+0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.9+0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.8+0.j,
         0. +0.j, -0.7+0.j, -0.6+0.j,  0.4+0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.6+0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0.4+0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.1+0.j,
        -1.5+0.j,  1.5+0.j,  0. +0.j,  0.6+0.j],
       [-0.6+0.j, -0.9+0.j,  0.8+0.j,  0.6+0.j, -0.1+0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [-0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  1.5+0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [-0.3+0.j,  0. +0.j,  0.7+0.j,  0. +0.j, -1.5+0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0.6+0.j,  0. +0.j,  0. +0.j,  0. +0

In [9]:
from pennylane.labs.dla import structure_constants_dense

all_mats = -1j * np.stack([matrix_map[pw] for pw in dla])
print(all_mats.shape)
for i in range(len(all_mats)):
    if not np.allclose(all_mats[i].conj().T, all_mats[i]):
        print(all_mats[i])
adj_pw = qml.structure_constants(dla)
adj_mat = structure_constants_dense(all_mats) * n_so
print(np.allclose(adj_pw, adj_mat))
print(np.allclose(np.abs(adj_pw), np.abs(adj_mat)))
for i in range(len(adj_mat)):
    if not np.allclose(adj_mat[i], adj_pw[i]):
        ids = np.where(~(adj_mat[i]==adj_pw[i]))
        print(i, ids)
        # print(adj_mat[i][ids])
        # print(adj_pw[i][ids])
        # print()
        # break

(45, 10, 10)
True
True
